In [1]:
import xml.etree.ElementTree as ET
import pandas as pd
import numpy as np
import geopandas as gpd
import folium
from glob import glob
from tqdm import tqdm
import pickle

In [2]:
import yaml
config_path = '/home/tu/tu_tu/tu_zxmav84/DS_Project/modules/config.yml'
with open(config_path, 'r') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)
city_3d_model_path = config['data']['city_3d_model']

In [3]:
with open(city_3d_model_path + 'processed/processed_roofs.pkl', 'rb') as f:
    gdf = pickle.load(f)

In [4]:
# See documentation of LOD2 here: https://www.ldbv.bayern.de/file/pdf/6723/Kundeninformation_LoD2.pdf
roof_types = {1000:'Flachdach', 3100:'Satteldach', 2100:'Pultdach', 3200:'Walmdach', 3300:'Walmdach'}
gdf['official_rooftype'] = gdf.roofType.map(roof_types)
gdf['official_rooftype'] = np.where(gdf.official_rooftype.isna(), 'Other', gdf.official_rooftype)

In [5]:
# def roof_type(x):
#     if all(value == 90 for value in x):
#         return 'flat'
#     elif any(value == 90 for value in x):
#         return 'mixed'
#     else:
#         return 'gable'

# gdf['type'] = gdf['Dachneigung'].apply(roof_type)

In [6]:
# make is much smaller for plotting
from shapely.geometry import box
bbox = box(*[11.557617,48.123591,11.599846,48.147649])
gdf = gdf[gdf.geometry_4326.intersects(bbox)]

In [7]:
# Constructing the convex hull for the collection of points
# gdf['convex_hull'] = gdf.geometry_4326.convex_hull

In [15]:
styles_dict = {"Flachdach":{"fillColor": "#ff0000", 'color':'#ff0000', 'weight':0, 'fillOpacity':0.7},
"Satteldach":{"fillColor": "#0000ff", 'color':'#0000ff', 'weight':0, 'fillOpacity':0.7},
"Pultdach":{"fillColor": "#00ff00", 'color':'#00ff00', 'weight':0, 'fillOpacity':0.7},
"Walmdach":{"fillColor": "#3C3C3C", 'color':'#3C3C3C', 'weight':0, 'fillOpacity':0.7},
"Other" :{"fillColor": "#000000", 'color':'#000000', 'weight':0, 'fillOpacity':0.7}}

m = folium.Map(location=[gdf.iloc[0,:].geometry_4326.centroid.y, gdf.iloc[0,:].geometry_4326.centroid.x], zoom_start=18)
tile = folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri', name = 'Esri Satellite', overlay = False, control = True).add_to(m)
# add roof types
for label in gdf.official_rooftype.unique():
        folium.GeoJson(gdf[gdf.official_rooftype == label].geometry_4326, style_function=lambda x, label=label: styles_dict[label], name = label).add_to(m)
folium.LayerControl().add_to(m)
m.save('roof_types.html')
#m